In [21]:
import os
import pandas as pd
import pymssql
from IPython.display import clear_output

In [22]:
# %%
transaction_code = pd.read_excel("D:/NCTU/10-資料範例與欄位說明/NCTU_AI_AML資料表欄位說明_v20210507.xlsx", sheet_name = '交易代碼表')
credit_code = transaction_code[transaction_code['交易方向']=='CREDIT']['交易代碼'].values.astype(int)
debit_code = transaction_code[transaction_code['交易方向']=='DEBIT']['交易代碼'].values.astype(int)

In [23]:
conn = pymssql.connect(server='.', user='NCTUUser2', password='NCTU@MetaEdge', database='NCTU(1090401-1091231)')
cursor = conn.cursor() 

In [24]:
"""
fetch the corresponding Txn_Type_Key according to the scenario
"""
def transactionKeySAR(scenario_name): 
    cursor.execute("select [Txn_Type_Key] from [dbo].[SAR_Info_TxnTypeKey] where [Scenario_name] = (%s)", (scenario_name))
    txnKey = []
    
    for row in cursor:
        txnKey.append(int(row[0]))
        #print(row[0])
    return txnKey

In [1]:
end_of_file = '_txn.csv'

In [12]:
"""
Txn aggregation according to the scenario and its corresponding detected days

df: Xday.csv
day: format "day5", "day15"..., this scenario check past X day txn record
scenario_name: aggregated txn file regarding this scenario
party: ACC/PTY
feature: feature that this scenario will used
"""
FEATURE = ['Credit_Amt', 'number_of_Credit', 'Debit_Amt','number_of_Debit', 'debit/credit']
def aggregation(df, day, scenario_name, party, feature, path, save_path):
    print(scenario_name + "txn aggregation... start!")
    data = []
    txnKey = transactionKeySAR(scenario_name)
    
    for i in range(df.shape[0]):
        data_slice = []
        
        if i%1000==0:
            clear_output(wait=True)

        txn_file_path = path + day + '/' + str(i) + end_of_file
        print('row account:', i, 'file path: ', txn_file_path)
        print('scenario name:', df['scenario_name'][i], 'corresponding txn key:', txnKey)
        
        if os.path.isfile(txn_file_path):
            txn_file = pd.read_csv(txn_file_path)
            if txn_file.empty is False:
                credit_idx = [] 
                debit_idx = []

                for j in range(txn_file.shape[0]):
                    if party=='ACC':
                         # make sure this transaction made by this primary_entity_level_number and this is a successful transaction
                        if txn_file['Acct_No'][j] == df['Acct_No'][i] and txn_file['Txn_Sts_Key'][j]==1:
                            # make sure this transaction type is in this scenario's specified scenario
                            if txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in credit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                credit_idx.append(j)
                            elif txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in debit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                debit_idx.append(j)

                    elif party=='PTY':
                        # make sure this is a successful transaction
                        if txn_file['Txn_Sts_Key'][j]==1:
                             # make sure this transaction type is in this scenario's specified scenario
                            if txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in credit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                credit_idx.append(j)
                            elif txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in debit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                debit_idx.append(j)

                credit = txn_file.iloc[credit_idx]
                credit_amt = credit['Ccy_Amt'].sum()
                credit_num = credit.shape[0]

                debit = txn_file.iloc[debit_idx]
                debit_amt = debit['Ccy_Amt'].sum()
                debit_num = debit.shape[0]

                data_slice = [credit_amt, credit_num, debit_amt, debit_num]

                if credit_amt!=0:
                    data_slice.append(debit_amt/credit_amt)
                else:
                    data_slice.append(-1) #-1 means credit_amt=0

            else:
                print('{}_txn.csv is empty'.format(str(i)))
            print('slice:',data_slice)
            data.append(data_slice)
        else:
            print('{}_txn.csv is not exist'.format(str(i)))
            #print('slice:',data_slice)
            data.append(data_slice)
  
    
    df1 = pd.DataFrame(data = data , columns = FEATURE)
    usecol = [*list(df.columns)]
    df = pd.concat([df, df1], axis = 1)
    usecol = [*usecol, *feature]
    df = df[usecol]
    df = df[df['scenario_name']==scenario_name]
    df.to_csv(save_path + scenario_name + "_"+ day + "_with_aggregated_txn_scenario_txn_type_key_營業日.csv", index = False)
    print(scenario_name + "txn aggregation... done!")

In [35]:
"""
Txn aggregation according to the shared features {FEATURE} and its detected days we want to observe

df: Xday.csv
day: format "day5", "day15"..., this scenario check past X day txn record
scenario_name: aggregated txn file regarding this scenario
party: ACC/PTY
feature: feature that this scenario will used
"""

FEATURE = ['Credit_Amt', 'number_of_Credit', 'Debit_Amt','number_of_Debit', 'debit/credit']


def aggregationScenarioGroup(df, group, day, path, save_path):
    print("Group txn aggregation... start!")
    data = []
    for i in range(df.shape[0]):
        data_slice = []
        
        if i % 1000 == 0:
            clear_output(wait = True)

        txn_file_path = path + day + '/' + str(i) + end_of_file
        print('row account:', i, 'file path: ', txn_file_path)
        
        party = df['primary_entity_level_code'][i]
        txnKey = transactionKeySAR(df['scenario_name'][i])
        
        if os.path.isfile(txn_file_path):
            txn_file = pd.read_csv(txn_file_path)
            if txn_file.empty is False:
                credit_idx = [] 
                debit_idx = []
                
                for j in range(txn_file.shape[0]):
                    if party=='ACC':
                        # make sure this transaction made by this primary_entity_level_number and this is a successful transaction
                        if txn_file['Acct_No'][j] == df['Acct_No'][i] and txn_file['Txn_Sts_Key'][j]==1:
                            if txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in credit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                credit_idx.append(j)
                            elif txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in debit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                debit_idx.append(j)

                    elif party=='PTY':
                        if txn_file['Txn_Sts_Key'][j]==1:
                            if txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in credit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                credit_idx.append(j)
                            elif txn_file['Txn_Type_Key'][j] in txnKey and txn_file['Txn_Type_Key'][j] in debit_code:
                                print('txn type key:', txn_file['Txn_Type_Key'][j])
                                debit_idx.append(j)

                credit = txn_file.iloc[credit_idx]
                credit_amt = credit['Ccy_Amt'].sum()
                credit_num = credit.shape[0]

                debit = txn_file.iloc[debit_idx]
                debit_amt = debit['Ccy_Amt'].sum()
                debit_num = debit.shape[0]

                data_slice = [credit_amt, credit_num, debit_amt, debit_num]

                if credit_amt!=0:
                    data_slice.append(debit_amt/credit_amt)
                else:
                    data_slice.append(-1) #-1 means credit_amt=0

            else:
                print('{}_txn.csv is empty'.format(str(i)))
            print('slice:',data_slice)
            data.append(data_slice)
        else:
            print('{}_txn.csv is not exist'.format(str(i)))
            #print('slice:',data_slice)
            data.append(data_slice)
  
    
    df1 = pd.DataFrame(data = data , columns = FEATURE)
    df = pd.concat([df, df1], axis = 1)
    res = pd.DataFrame()

    for item in group:
        tmp = df[df['scenario_name']==item]
        res = res.append(tmp)
        
    res.to_csv(save_path + "scenarioGroup_" + day + "_with_aggregated_txn_scenario_txn_type_key_營業日.csv", index = False)
    print("Group txn aggregation... done!")